# Introduction
The author follows the CRISP-DM methodology.
![CRISP-DM](./data/crisp.png)

# Business Understanding

Many people struggle to get loans due to insufficient or non-existent credit histories. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

Home Credit strives to broaden financial inclusion for the unbanked population by providing a positive and safe borrowing experience. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

# Data Understanding

application_{train|test}.csv

- This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
- Static data for all applications. One row represents one loan in our data sample.

bureau.csv

- All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
- For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

bureau_balance.csv

- Monthly balances of previous credits in Credit Bureau.
- This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

POS_CASH_balance.csv

- Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
- This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

credit_card_balance.csv

- Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
- This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

previous_application.csv

- All previous applications for Home Credit loans of clients who have loans in our sample.
- There is one row for each previous application related to loans in our data sample.

installments_payments.csv

- Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
- There is a) one row for every payment that was made plus b) one row each for missed payment.
- One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

HomeCredit_columns_description.csv

- This file contains descriptions for the columns in the various data files.

![CRISP-DM](./data/home_credit.png)

Start Spark

In [1]:
import pandas as pd
import findspark 
import os
from pyspark import SparkContext 
from pyspark.sql import SparkSession
findspark.init() 
spark = SparkSession\
    .builder\
    .appName("HomeCreditApp")\
    .config("spark.driver.memory", "8G")\
    .config("spark.executor.memory", "4G")\
    .getOrCreate()

In [2]:
sparkApplicationData = spark\
            .read\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv("Data/application_train.csv")\
            .cache()

In [3]:
sparkApplicationDataTest = spark\
            .read\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv("Data/application_test.csv")\
            .cache()

In [4]:
sparkPosCashData = spark\
            .read\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv("Data/POS_CASH_balance.csv")\
            .cache()
sparkPosCashData = sparkPosCashData.withColumnRenamed("SK_ID_CURR","SK_ID_CURR_POS")
sparkPosCashData = sparkPosCashData.groupBy('SK_ID_CURR_POS').max()

Join tables

In [5]:
sparkApplicationData = sparkApplicationData.join(sparkPosCashData, sparkApplicationData.SK_ID_CURR == sparkPosCashData.SK_ID_CURR_POS, how='left')

In [6]:
sparkApplicationDataTest = sparkApplicationDataTest.join(sparkPosCashData, sparkApplicationDataTest.SK_ID_CURR == sparkPosCashData.SK_ID_CURR_POS, how='left')

Columns and null values

In [7]:
sparkApplicationData\
  .printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

In [8]:
from pyspark.sql.functions import isnull
from IPython.core.display import HTML
for c in sparkApplicationData.columns:
  nullCnt = sparkApplicationData.where(isnull(c)).count()
  display(HTML('Column <b>{}</b> has <b>{}</b> null records'.format(str(c), nullCnt))) 

In [9]:
def displayPandas(sparkData):
    display(
    sparkData
      .limit(5)
      .toPandas()
    )

First 5 rows.

In [10]:
displayPandas(sparkApplicationData)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SK_ID_CURR_POS,max(SK_ID_PREV),max(SK_ID_CURR_POS),max(MONTHS_BALANCE),max(CNT_INSTALMENT),max(CNT_INSTALMENT_FUTURE),max(SK_DPD),max(SK_DPD_DEF)
0,351232,0,Cash loans,F,N,N,0,180000.0,495000.0,25146.0,...,0.0,0.0,351232,2398104,351232,-7,18.0,18.0,12,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,100003,2636178,100003,-18,12.0,12.0,0,0
2,351234,0,Cash loans,F,N,Y,1,90000.0,107820.0,7798.5,...,0.0,2.0,351234,2835678,351234,-6,24.0,24.0,0,0
3,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,100007,2462742,100007,-1,24.0,24.0,0,0
4,373969,0,Cash loans,M,N,Y,0,202500.0,463284.0,22662.0,...,0.0,2.0,373969,1925229,373969,-10,12.0,12.0,0,0


# Data Preparation

Null values replace with average value.

In [11]:
## filter numeric cols
num_cols = [col_type[0] for col_type in filter(lambda dtype: dtype[1] in {"bigint", "double", "int"}, sparkApplicationData.dtypes)]
### Compute a dict with <col_name, median_value>
median_dict = dict()
for c in num_cols:
   median_dict[c] = sparkApplicationData.stat.approxQuantile(c, [0.5], 0.001)[0]

In [12]:
sparkApplicationData = sparkApplicationData.na.fill(median_dict)

In [13]:
sparkApplicationData.write\
        .format("com.databricks.spark.csv")\
        .option("header", "true")\
        .mode("overwrite")\
        .save('Data/preparedTrain.csv')\

# Modeling

Data is transfered to H20

In [14]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 hours 48 mins
H2O cluster timezone:,Europe/Prague
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,"2 years, 1 month and 1 day !!!"
H2O cluster name:,H2O_from_python_lukas_sykora_zfasx0
H2O cluster total nodes:,1
H2O cluster free memory:,2.149 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"locked, healthy"


In [15]:
df_train = h2o.import_file('Data/preparedTrain.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [16]:
predictors = df_train.drop('TARGET').names

In [17]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models=2, seed=42)
aml.train(
    x=predictors, 
    y='TARGET', 
    training_frame = df_train
)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Evaluation

The performance of models.

In [18]:
aml.leaderboard

model_id,mean_residual_deviance,rmse,mae,rmsle
StackedEnsemble_BestOfFamily_0_AutoML_20200407_004427,0.069497,0.263623,0.139002,0.184286
StackedEnsemble_AllModels_0_AutoML_20200407_004427,0.069497,0.263623,0.139002,0.184286
XRT_0_AutoML_20200407_004427,0.070111,0.264784,0.146925,0.186737
DRF_0_AutoML_20200407_004427,0.070123,0.264808,0.141587,0.186381


In [24]:
print("Training Data")
#aml.leader.model_performance(train = True).plot()
print("X-Val")
#aml.leader.model_performance(xval=True).plot()

Training Data
X-Val


In [25]:
try:
    aml.leader.varimp_plot(20)
except:
    print("No variable importance for this model")  

No variable importance for this model


# Deployment

Use the model for new data.

In [30]:
del median_dict['TARGET']
sparkApplicationDataTest = sparkApplicationDataTest.na.fill(median_dict)
sparkApplicationDataTest.write\
        .format("com.databricks.spark.csv")\
        .option("header", "true")\
        .mode("overwrite")\
        .save('Data/preparedTest.csv')
df_test = h2o.import_file('Data/preparedTest.csv')
pred = aml.leader.predict(df_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [33]:
pred = pred.as_data_frame()
pred.to_csv('Data/pred.csv', index=False) 

In [35]:
pred['Target'] = round(pred['predict'])

In [36]:
pred.to_csv('Data/predRound.csv', index=False) 

In [ ]:
spark.stop() #closing the spark session